# Темы от меня
### 1. Рассказ про метрики

- В том числе и про метрики в ранжировании precision@k, recall@k, AP@k, MAP@k, nDCG

### 2. ROG-AUG 
У доски разобрать [материалы][1]:

- формулу 
- как стоим Рог Кривую  
- можем попробовать к ней (задаче в посте) построить PR-curve

[1]: https://alexanderdyakonov.wordpress.com/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/

# Тема: Метрики 

### План:
1. Бизнес цель 
2. Общий взгляд оптимизации по данным (блумрег)
3. Оценки алгоритма (в рамках выбранных признаков и данных)
4. Оценка модели (метрики) (модель = (алгоритм, признаки, данные) в рамках бизнес постановки) (offline/Online(?) = ML метрики)
5. Оценка сервиса (метрики MLops, DevOps)
6. Оценка бизнесом собранного инструмента (иерархия бизнес метрик) (была книга)

## 1. Бизнес
Много разных целей на разных этапах развития проекта.
- выигрыш сейчас,
- выигрыш в долгую,
- рост клиентов,
- рост стоимости (за счет AI в буклетах),

на разных этапах можно проигрывать в деньгах (еще нет клиентов, но есть расходы на разработку и продвижение ... )

## 2. Оптимизация
[лекции от блумберг][1] = лекция 3 и 5 

[1]: https://bloomberg.github.io/foml/#lectures



Риск определяется как:

$$ R(f) = \mathbb{E} l(f(x),y)  $$

Ожидаемая потеря на новых данных случайно взятых из некоторого распределения данных.

В действительности мы не знаем реальное распределение данных, но мы можем оценить на тех данных, что у нас есть.

**The Empirical Risk Functional**

Пусть у нас есть $data = ((x_1, y_1), ..., (x_n, y_n))$

$$ \hat{R}(f) = \frac{1}{n} \sum_{i=1}^{n} l(f(x_i),y_i)  $$


**Empirical Risk Minimization**

$ \hat{f} $ - оптимальное решение для эмпирического риска, когда 

$$ \hat{f} = \argmin_{f} \hat{R}_n(f)$$


Пример. 

Пусть X задано на [0,1], Y ≡ 1 (Y всегда 1). Допустим, что $data = ((0.25, 1), (0.5,1), (0.75, 1))$

Пусть 
$ \hat{f}  = 1(x \in {0.25, 0.5, 0.75})$ 

Что тогда мы получаем?

Эмпирический риск для $ \hat{f} = 0$, но сам риск = 1. 
 

Для лучшего обобщения вводят поиск не на всем множестве функции, а на некоторых наборах функциях. Эти наборы функций и определяют разные методы ML.


![image.png](../08/Error_decomposition.jpg)

## 3. Оценка алгоритма (Функция потерь)

### 3.1 Для задач регрессии


#### 3.1.1 MSE

Оптимальное решение = среднее (как доказать?)

In [2]:
def MSE(ax,y):
    if len(ax)!= len(y):
        print("len(ax)!= len(y)")
        return -1
    
    res = 0
    for i in range(len(y)):
        res+= (y[i]-ax[i])**2
    return res

Домашнее задание:
1. Придумать однострочник вида `return sum ()`.
2. Как реализовать эффективно через Numpy?

#### 3.1.2 MAE
Оптимальное решение = медиана 

In [1]:
def MAE(ax,y):
    if len(ax)!= len(y):
        print("len(ax)!= len(y)")
        return -1
    
    res = 0
    for i in range(len(y)):
        res+= abs(y[i]-ax[i])
    return res

MAE vs MSE

In [3]:
# Примеры
# target
y = [100]*10
# model 1 
ax1 = [100]*9 + [900]
# model 2 
ax2 = [200]*10

In [4]:
MSE(ax1, y)

640000

In [5]:
MAE(ax1, y)

800

In [6]:
MSE(ax2, y)

100000

In [7]:
MAE(ax2, y)

1000

#### 3.1.3 MAPE, sMAPE, wMAPE 
Придумать примеры, когда лучше одна или другая?

### 3.2 Для задач классификации

https://proproprogs.ru/ml/ml-funkcii-poter-v-zadachah-lineynoy-binarnoy-klassifikacii

<!-- ![image.png](../08/bin_class_loss.png) -->
<img src="../08/bin_class_loss.png" alt="drawing" width="700"/>

 - $V(M) = (1-M)_+$ - кусочно-линейная (SVM);
 - $H(M) = (-M)_+$ - кусочно-линейная (Hebb’s rule);
 - $L(M) = log_2(1+ e^{-M})$ -  логарифмическая (LR);
 - $Q(M) = (1-M)^2$ - квадратичная (FLD);
 - $S(M) = 2 \cdot (1 + e^{M})^{-1}$ - сигмоидная (ANN);
 - $E(M) = e^{-M}$ - экспоненциальная (AdaBoost).

Особенность всех этих функций (помимо того, что они всюду дифференцируемы) в том, что они по значению больше, либо раны исходной пороговой. Следовательно, минимизируя любую из них, мы автоматически будем минимизировать потери и при пороговой функции.

### 3.3 Для задач ранжирования
#### 3.3.1 
- [PairwiseLogisticLoss][1]
- [PairwiseHingeLoss][2]
- [PairwiseMSELoss][3]

[1]: https://www.tensorflow.org/ranking/api_docs/python/tfr/keras/losses/PairwiseLogisticLoss
[2]: https://www.tensorflow.org/ranking/api_docs/python/tfr/keras/losses/PairwiseHingeLoss
[3]: https://www.tensorflow.org/ranking/api_docs/python/tfr/keras/losses/PairwiseMSELoss

## 4. Метрики модели (как оцениваем модель)




### 4.1 Для задач регрессии
#### 4.1.1 



### 4.2 Для задач классификации
#### 4.2.1 Accuracy, Precision, Recall
#### 4.2.2 ROG-AUG, PR-curve

### 4.3 Для задач ранжирования
#### 4.3.1 

## 5. Оценка сервиса (метрики MLops, DevOps)


## 6. Оценка бизнесом собранного инструмента (иерархия метрик) 

[Продуктовые метрики: какие они бывают и как ими пользоваться?][1]
<!-- ![](../08/Product_metrics.webp) -->
<img src="../08/Product_metrics.webp" alt="drawing" width="700"/>


[1]: https://vc.ru/marketing/310152-produktovye-metriki-kakie-oni-byvayut-i-kak-imi-polzovatsya

[Базовые продуктовые метрики][2]

[2]: https://vc.ru/marketing/314555-bazovye-produktovye-metriki


(была книга)
[Metrics Pyramid and Metrics Hierarchy][3]

[3]: https://imaga.co/blog/analytics/metrics-pyramid-perhaps-the-best-way-to-understand-what-s-wrong-with-your-product/

<!-- ![](../08/Business_metrics.webp) -->
<img src="../08/Business_metrics.webp" alt="drawing" width="1200"/>

<!-- ![](../08/Business_metrics.webp) -->
<img src="../08/Business_metrics2.webp" alt="drawing" width="1200"/>